# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import FileDatasetFactory
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from automl_data_utils import get_image_data_as_df, convert_to_table

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'covid-pneumonia-automl'

experiment=Experiment(ws, experiment_name)

compute_name = "compute-cluster1"
vm_size = "STANDARD_NC6"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print("Found existing cluster, using it!")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=6)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion()

Found existing cluster, using it!


In [3]:
# train_data, test_data = get_image_data_as_df(ws, './Covid19-dataset/train', './Covid19-dataset/test',"covid-pneumonia-dataset-table")

test_data = convert_to_table('./Covid19-dataset/test')

Found 66 images belonging to 3 classes.
Type: float64


In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12991,12992,12993,12994,12995,12996,12997,12998,12999,class
0,0.000000,0.007843,0.039216,0.066667,0.078431,0.101961,0.109804,0.117647,0.125490,0.211765,...,0.396078,0.349020,0.125490,0.074510,0.007843,0.000000,0.000000,0.000000,0.023529,2
1,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,...,0.250980,0.184314,0.066667,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0
2,0.000000,0.000000,0.000000,0.952941,0.364706,0.207843,0.231373,0.266667,0.286275,0.588235,...,0.082353,0.082353,0.082353,0.082353,0.082353,0.082353,0.000000,0.000000,0.000000,1
3,0.090196,0.101961,0.145098,0.168627,0.203922,0.513726,0.384314,0.372549,0.376471,0.643137,...,0.000000,0.000000,0.000000,0.011765,0.019608,0.023529,0.015686,0.023529,0.027451,1
4,0.031373,0.039216,0.027451,0.031373,0.023529,0.031373,0.035294,0.019608,0.019608,0.031373,...,0.462745,0.188235,0.074510,0.015686,0.023529,0.035294,0.039216,0.039216,0.043137,1


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_factory = TabularDatasetFactory.register_pandas_dataframe(train_data, target=ws.get_default_datastore(), name='CovidPneumoniaTable')

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/cd122184-bb30-4220-9254-f5949faad286/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"enable_dnn":True,
                   "experiment_timeout_minutes":45,
                   "max_cores_per_iteration":3,
                   "enable_tf":False,
                   "featurization":"off"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             compute_target=compute_cluster,
                             training_data=data_factory,
                             label_column_name='class',
                             **automl_settings
                             )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
covid-pneumonia-automl,AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec',
 'target': 'compute-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2021-04-17T05:17:17.456526Z',
 'endTimeUtc': '2021-04-17T06:41:09.833316Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster1',
  'AMLSettingsJsonString': '{"path":null,"name":"covid-pneumonia-automl","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-142902","workspace_name":"quick-starts-ws-142902","region":"southcentralus","compute_target":"compute-cluster1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
#TODO: Save the best model
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()
best_run_properties = best_run_parameters['properties']

print("Best Run:",best_run.id)
print("Algorithm:", best_run_properties['run_algorithm'])
print("Metric:", best_run_properties['primary_metric'])
print("Best Score:",best_run_properties['score'])


Best Run: AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec_12
Algorithm: VotingEnsemble
Metric: accuracy
Best Score: 0.9563076923076924


In [10]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/65_job_prep-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/94_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [4]:
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice

# registered_model = best_run.register_model(model_name='covid-pneumonia-automl-model',
#                                             model_path='./outputs/model.pkl')

registered_model = Model.register(workspace = ws,
                        model_path ="./AutoMLmodel/model.pkl",
                        model_name = "covid-pneumonia-automl-model",
                        description = "AutoML Covid-Pneumonia Classifier")

Registering model covid-pneumonia-automl-model


In [5]:
env = Environment.from_conda_specification(name='deploy-env', file_path='./deploy_dependencies.yml')

In [6]:
inference_config = InferenceConfig(entry_script='score_automl.py',
                                    environment=env)

In [7]:
# Create a local deployment, using port 8890 for the web service endpoint
deployment_config = LocalWebservice.deploy_configuration(port=8890)

# Deploy the service
service = Model.deploy(
    ws, "mymodel2", [registered_model], inference_config, deployment_config)

# Wait for the deployment to complete
service.wait_for_deployment(True)

# Display the port that the web service is available on
print(service.port)

Generating Docker build context.
2021/04/17 15:27:20 Downloading source code...
2021/04/17 15:27:22 Finished downloading source code
2021/04/17 15:27:22 Creating Docker network: acb_default_network, driver: 'bridge'
2021/04/17 15:27:22 Successfully set up Docker network: acb_default_network
2021/04/17 15:27:22 Setting up Docker configuration...
2021/04/17 15:27:23 Successfully set up Docker configuration
2021/04/17 15:27:23 Logging in to registry: 2dc30e120150428f9f51e9a661d86a4e.azurecr.io
2021/04/17 15:27:24 Successfully logged into 2dc30e120150428f9f51e9a661d86a4e.azurecr.io
2021/04/17 15:27:24 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/04/17 15:27:24 Scanning for dependencies...
2021/04/17 15:27:25 Successfully scanned dependencies
2021/04/17 15:27:25 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2021030


done
#
# To activate this environment, use
#
#     $ conda activate /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339
#
# To deactivate an active environment, use
#
#     $ conda deactivate



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda


Removing intermediate container b2507f01be8c
 ---> f12e1eb802c3
Step 9/18 : ENV PATH /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/bin:$PATH
 ---> Running in 1bcb5b1fbb44
Removing intermediate container 1bcb5b1fbb44
 ---> b227d07336bd
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> baee6bd707ab
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> 6ced4c9bde34
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/

Package creation Succeeded
Logging into Docker registry 2dc30e120150428f9f51e9a661d86a4e.azurecr.io
Logging into Docker registry 2dc30e120150428f9f51e9a661d86a4e.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 2dc30e120150428f9f51e9a661d86a4e.azurecr.io/azureml/azureml_7116654fd6c139721b7b847f9045cbf7
 ---> 710f8efa60d0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 2f853a468d36
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImI5NjhmYjM2LWYwNmEtNGM3Ni1hMTVmLWFmYWI2OGFlNzY2NyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE0Mjk0OSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE0Mjk0OSIsIndvcmtzcGFjZUlkIjoiMmRjMzBlMTItMDE1MC00MjhmLTlmNTEtZTlhNjYxZDg2YTRlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 94f24569fdd8
 ---> 92d4658f9980
Step 4/5 : RUN mv '/var/azureml-app/tmpyzq5mc7b.py' /var/azureml-app/main.py
 ---> Running in f09215279cf6
 --

In [8]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-04-17T15:36:29,007411120+00:00 - iot-server/run 
2021-04-17T15:36:29,007731821+00:00 - rsyslog/run 
2021-04-17T15:36:29,008053821+00:00 - gunicorn/run 
bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by bash)
2021-04-17T15:36:29,009262624+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libcrypto.so.1.0.0

In [9]:
test_data_small = test_data.head(10)
labels = test_data_small['class']
test_data_small = test_data_small.drop(['class'], axis=1)
test_data_small

,0,1,2,3,4,5,6,7,8,9,...,12990,12991,12992,12993,12994,12995,12996,12997,12998,12999
0,0.003922,0.000000,0.000000,0.000000,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011765,0.000000,0.015686,0.000000,0.000000,0.000000,0.000000,0.058824,0.149020,0.803922
1,0.180392,0.125490,0.109804,0.996078,0.882353,0.925490,0.925490,0.145098,0.156863,0.160784,...,0.305882,0.219608,0.109804,0.117647,0.129412,0.141176,0.145098,0.149020,0.172549,0.203922
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.905882,0.901961,0.913726,0.847059,0.831373,0.835294,0.800000,0.760784,0.658824,0.490196
3,0.517647,0.627451,0.584314,0.631373,0.576471,0.470588,0.396078,0.341176,0.113725,0.141176,...,0.054902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.592157,0.588235,0.588235,0.588235,0.588235,0.603922,0.188235,0.184314,0.184314,0.188235,...,0.349020,0.345098,0.337255,0.317647,0.313726,0.309804,0.294118,0.290196,0.282353,0.298039
5,0.000000,0.000000,0.000000,0.603922,0.619608,0.839216,0.756863,0.713726,0.701961,0.745098,...,0.188235,0.105882,0.007843,0.027451,0.050980,0.082353,0.082353,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.180392,0.125490,0.141176,0.168627,0.152941,0.192157,0.686275,...,0.564706,0.435294,0.674510,0.698039,0.635294,0.992157,0.996078,0.000000,0.000000,0.000000
7,0.184314,0.533333,0.223529,0.168627,0.156863,0.149020,0.141176,0.109804,0.129412,0.129412,...,0.443137,0.580392,0.549020,0.109804,0.066667,0.050980,0.015686,0.019608,0.023529,0.015686
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.709804,0.686275,0.674510,0.658824,0.647059,0.600000,0.670588,0.709804,0.690196,0.560784
9,0.011765,0.011765,0.015686,0.007843,0.015686,0.011765,0.015686,0.007843,0.011765,0.011765,...,0.678431,0.537255,0.490196,0.352941,0.458824,0.078431,0.011765,0.011765,0.011765,0.011765


In [19]:
import json

dictData = test_data_small.to_dict()

input_payload = json.dumps({
    'data': dictData
})

# print(input_payload)

service.reload()
output = service.run(input_payload)

print(output)

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
Checking container health...
Local webservice is running at http://localhost:8890
[0, 0, 0, 2, 0, 1, 2, 2, 0, 0]


In [20]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by bash)
/usr/sbin/nginx: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libcrypto.so.1.0.0: no version information available (required by

In [21]:
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice

registered_model = best_run.register_model(model_name='covid-pneumonia-automl-model',
                                            model_path='./outputs/model.pkl')

In [29]:
env = Environment.from_conda_specification(name='deploy-env', file_path='./deploy_dependencies.yml')

In [30]:
inference_config = InferenceConfig(entry_script='score_automl.py',
                                    environment=env)

In [31]:
# Create a local deployment, using port 8890 for the web service endpoint
deployment_config = LocalWebservice.deploy_configuration(port=8890)

# Deploy the service
service = Model.deploy(
    ws, "mymodel2", [registered_model], inference_config, deployment_config)

# Wait for the deployment to complete
service.wait_for_deployment(True)

# Display the port that the web service is available on
print(service.port)

Generating Docker build context.
2021/04/17 08:08:09 Downloading source code...
2021/04/17 08:08:10 Finished downloading source code
2021/04/17 08:08:10 Creating Docker network: acb_default_network, driver: 'bridge'
2021/04/17 08:08:11 Successfully set up Docker network: acb_default_network
2021/04/17 08:08:11 Setting up Docker configuration...
2021/04/17 08:08:12 Successfully set up Docker configuration
2021/04/17 08:08:12 Logging in to registry: 0b482f80b86f4e77a786cb9ab857ea42.azurecr.io
2021/04/17 08:08:13 Successfully logged into 0b482f80b86f4e77a786cb9ab857ea42.azurecr.io
2021/04/17 08:08:13 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/04/17 08:08:13 Scanning for dependencies...
2021/04/17 08:08:14 Successfully scanned dependencies
2021/04/17 08:08:14 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2021030

Removing intermediate container 5b501a40af4b
 ---> d2aba3e7e3d3
Step 9/18 : ENV PATH /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/bin:$PATH
 ---> Running in 9a13837d8edb
Removing intermediate container 9a13837d8edb
 ---> e264fda98147
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> a17cc8c988e2
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> f1587d99bc73
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339
 ---> Running in ecc9d7838318
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with materialized conda environment details
Successfully sent materialized environment details
Removing intermediate container ecc9d7838318
 ---> b98fd80a791b
Step 13/18 : ENV AZUREML

In [32]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-04-17T08:16:29,309488295+00:00 - gunicorn/run 
2021-04-17T08:16:29,309823796+00:00 - iot-server/run 
2021-04-17T08:16:29,311133901+00:00 - nginx/run 
2021-04-17T08:16:29,312153904+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16c339/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a65aa3b99d7ae787d24a55632e16

In [22]:
aciConfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
#                                                tags={'name': 'covid-pneumonia', 'framework': 'Keras'},
                                               description='Covid and Pneumonia Recognition using AutoML')

aci_service_name = "my-service"
aci_service = Model.deploy(ws, aci_service_name, [registered_model], inference_config=inference_config, deployment_config=aciConfig)
aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-17 16:15:13+00:00 Creating Container Registry if not exists.
2021-04-17 16:15:13+00:00 Registering the environment.
2021-04-17 16:15:15+00:00 Use the existing image.
2021-04-17 16:15:15+00:00 Generating deployment configuration.
2021-04-17 16:15:16+00:00 Submitting deployment to compute.
2021-04-17 16:15:23+00:00 Checking the status of deployment my-service..
2021-04-17 16:17:30+00:00 Checking the status of inference endpoint my-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
import json

dictData = test_data_small.to_dict()

input_payload = json.dumps({
    'data': dictData
})

# print(input_payload)

# aci_service.reload()
output = aci_service.run(input_payload)

print(output)

[0, 0, 0, 2, 0, 1, 2, 2, 0, 0]


TODO: In the cell below, print the logs of the web service and delete the service